# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [6]:
import os 
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-1106",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [7]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

## Prepare the Langchain dataframe agent

In [8]:
df.sample(5)

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,negative,negativeIncrease,negativeTestsAntibody,negativeTestsPeopleAntibody,negativeTestsViral,onVentilatorCumulative,onVentilatorCurrently,positive,positiveCasesViral,positiveIncrease,positiveScore,positiveTestsAntibody,positiveTestsAntigen,positiveTestsPeopleAntibody,positiveTestsPeopleAntigen,positiveTestsViral,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
1273,2021-02-13,PA,23072.0,0.0,113,0.0,0.0,0.0,2452.0,0,0.0,489.0,3757904.0,13578,0.0,0.0,0.0,0.0,271.0,892344.0,771452.0,4088,0,0.0,0.0,0.0,0.0,0.0,776339.0,9799882.0,72154,9799882.0,72154,0.0,0.0,0.0,0.0,4529356.0,16945,0.0,0
19560,2020-03-23,IL,12.0,0.0,3,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1273.0,0.0,224,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9868.0,1494,0.0,0.0,0.0,0.0,0.0,0,9868.0,1494
1594,2021-02-07,MO,7143.0,0.0,1,0.0,0.0,0.0,1637.0,0,0.0,348.0,1799152.0,3234,120119.0,0.0,3725886.0,0.0,210.0,466217.0,466217.0,769,0,21351.0,74876.0,0.0,0.0,514614.0,0.0,0.0,0,4249531.0,11676,141657.0,701782.0,128051.0,300966.0,2265369.0,4003,4249531.0,11676
6881,2020-11-05,VA,3688.0,3426.0,11,262.0,12865.0,12865.0,1064.0,68,0.0,240.0,0.0,0,0.0,0.0,0.0,0.0,104.0,187202.0,172418.0,1366,0,11466.0,10584.0,0.0,0.0,205694.0,0.0,2719206.0,18441,2719206.0,18441,154933.0,178110.0,0.0,0.0,0.0,0,0.0,0
8936,2020-09-29,NE,472.0,0.0,2,0.0,2301.0,2301.0,214.0,21,0.0,0.0,407979.0,3681,0.0,0.0,556572.0,0.0,0.0,44578.0,0.0,515,0,0.0,0.0,0.0,0.0,53392.0,33087.0,0.0,0,611080.0,4238,0.0,0.0,0.0,0.0,452862.0,4196,611080.0,4238


In [9]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To determine the number of rows in the dataframe, I would typically use the `.shape` attribute or the `len()` function on the dataframe.

Action: python_repl_ast
Action Input: len(df)
Observation: 20780
Thought:I now know the final answer.
Final Answer: There are 20780 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20780 rows in the dataframe.'}

## Design your prompt and ask your question

In [10]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2020" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: First, I need to set the pandas display options to show all columns. Then, I will retrieve the column names from the dataframe.
Action: Execute the Python command to set the display option for maximum columns to None, which means there is no maximum limit.
Action Input: pd.set_option('display.max_columns', None)
Observation: Execute the Python command to set the display option for maximum columns to None, which means there is no maximum limit. is not a valid tool, try one of [python_repl_ast].
Thought:My previous action was incorrect. I need to use the python_repl_ast tool to execute the Python command.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)
Observation: NameError: name 'pd' is not defined
Thought:I need to import pandas as pd before setting the display option.
Action: python_repl_ast
Action Input: import pandas as pd; pd.set_option('display.max_columns', None)
Observation: 
Thought:The pandas di

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, 

In [11]:
QUESTION = "How may patients were recovered during July 2020" 
"in Texas, and nationwide as the total of all states?"
"Use the recovered column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: First, I need to set the pandas display options to show all columns.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
```
Observation: 
Thought:Now that the pandas display options are set to show all columns, I should get the column names of the dataframe.
Action: python_repl_ast
Action Input: df.columns.tolist()
Observation: ['date', 'state', 'death', 'deathConfirmed', 'deathIncrease', 'deathProbable', 'hospitalized', 'hospitalizedCumulative', 'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative', 'inIcuCurrently', 'negative', 'negativeIncrease', 'negativeTestsAntibody', 'negativeTestsPeopleAntibody', 'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently', 'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore', 'positiveTestsAntibody', 'positiveTestsAntigen', 'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen', 'positiveTe

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were recovered during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, men